In [2]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 KB 9.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.4/220.4 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.1/206.1 KB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.3/327.3 KB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 KB 34.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling

In [8]:
# import json
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.feature_extraction.text import TfidfVectorizer
# import re
# from typing import List, Dict, Tuple
# import logging
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# import seaborn as sns
# from pathlib import Path
# import urllib.request

# # Set up logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# logger = logging.getLogger(__name__)

# # Set seaborn style for better visuals
# sns.set(style="whitegrid")

# # Check for bitsandbytes availability
# try:
#     import bitsandbytes
#     logger.info(f"bitsandbytes version: {bitsandbytes.__version__}")
#     use_quantization = True
# except ImportError:
#     logger.warning("bitsandbytes not found. Running without 4-bit quantization.")
#     use_quantization = False

# # Initialize Qwen3-0.6B model and tokenizer
# model_name = "Qwen/Qwen3-0.6B"
# try:
#     tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#     model_kwargs = {
#         "device_map": "auto",
#         "trust_remote_code": True
#     }
#     if use_quantization:
#         model_kwargs["load_in_4bit"] = True
#     model = AutoModelForCausalLM.from_pretrained(model_name, **model_kwargs)
#     model.eval()
# except Exception as e:
#     logger.error(f"Error loading model: {e}")
#     raise

# # Load MenatQA dataset
# def load_menatqa_dataset(file_path='./MenatQA.json'):
#     file_path = Path(file_path)
    
#     if not file_path.exists():
#         logger.info(f"Downloading MenatQA to {file_path}...")
#         try:
#             urllib.request.urlretrieve(
#                 "https://raw.githubusercontent.com/weiyifan1023/MenatQA/main/datasets/MenatQA.json",
#                 str(file_path)
#             )
#         except Exception as e:
#             logger.error(f"Failed to download MenatQA: {e}")
#             raise
    
#     with open(file_path, 'r') as f:
#         data = json.load(f)
#     logger.info(f"Loaded MenatQA dataset with {len(data)} examples")
#     return data

# # Extract reasoning hops from model output
# def extract_hops(response: str) -> List[str]:
#     hop_pattern = r'<think>(.*?)</think>|(\d+\.\s*[^.]+)'
#     hops = re.findall(hop_pattern, response, re.DOTALL)
#     hops = [hop[0] if hop[0] else hop[1].strip() for hop in hops]
#     return [hop for hop in hops if hop]

# # Generate forward reasoning
# def forward_reasoning(question: str, context: str) -> Tuple[str, List[str]]:
#     prompt = f"Context: {context}\nQuestion: {question}\nAnswer the question step-by-step, showing your reasoning."
#     messages = [{"role": "user", "content": prompt}]
#     try:
#         text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=True)
#         inputs = tokenizer(text, return_tensors="pt").to(model.device)
#         with torch.no_grad():
#             response_ids = model.generate(**inputs, max_new_tokens=4096)[0][len(inputs.input_ids[0]):]
#         response = tokenizer.decode(response_ids, skip_special_tokens=True)
#         answer = response.split("Final Answer:")[-1].strip() if "Final Answer:" in response else response.strip().split('\n')[-1]
#         hops = extract_hops(response)
#         return answer, hops
#     except Exception as e:
#         logger.error(f"Error in forward reasoning: {e}")
#         return "", []

# # Generate backward reasoning
# def backward_reasoning(question: str, context: str, answer: str) -> List[str]:
#     prompt = f"Context: {context}\nThe answer to the question '{question}' is '{answer}'. What context information and reasoning steps led to this answer? Provide the steps in reverse order."
#     messages = [{"role": "user", "content": prompt}]
#     try:
#         text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=True)
#         inputs = tokenizer(text, return_tensors="pt").to(model.device)
#         with torch.no_grad():
#             response_ids = model.generate(**inputs, max_new_tokens=4096)[0][len(inputs.input_ids[0]):]
#         response = tokenizer.decode(response_ids, skip_special_tokens=True)
#         return extract_hops(response)
#     except Exception as e:
#         logger.error(f"Error in backward reasoning: {e}")
#         return []

# # Check consistency between forward and backward hops
# def check_consistency(forward_hops: List[str], backward_hops: List[str]) -> bool:
#     if not forward_hops or not backward_hops:
#         return False
#     reversed_forward = forward_hops[::-1]
#     if len(forward_hops) != len(backward_hops):
#         return False
#     vectorizer = TfidfVectorizer()
#     all_hops = forward_hops + backward_hops
#     tfidf_matrix = vectorizer.fit_transform(all_hops)
#     forward_vectors = tfidf_matrix[:len(forward_hops)]
#     backward_vectors = tfidf_matrix[len(forward_hops):]
#     similarities = [cosine_similarity(forward_vectors[i], backward_vectors[i])[0][0] for i in range(len(forward_hops))]
#     avg_similarity = np.mean(similarities)
#     return avg_similarity > 0.85

# # Generate transitive closure and check compliance
# def transitivity_check(hops: List[str]) -> Tuple[List[str], float]:
#     inferred_facts = []
#     for i in range(len(hops)):
#         for j in range(i + 1, len(hops)):
#             # Numeric transitivity
#             if " = " in hops[i] and "> " in hops[j]:
#                 value_match = re.search(r'(\w+)\s*=\s*(\d+)', hops[i])
#                 comparison_match = re.search(r'(\w+)\s*>\s*(\d+)', hops[j])
#                 if value_match and comparison_match and value_match.group(1) == comparison_match.group(1):
#                     inferred_facts.append(f"{value_match.group(1)} > {comparison_match.group(2)}")
#             # Temporal transitivity
#             if "before" in hops[i] and "after" in hops[j]:
#                 before_match = re.search(r'(\w+)\s+before\s+(\w+)', hops[i])
#                 after_match = re.search(r'(\w+)\s+after\s+(\w+)', hops[j])
#                 if before_match and after_match and before_match.group(2) == after_match.group(1):
#                     inferred_facts.append(f"{before_match.group(1)} before {after_match.group(2)}")
#     if not inferred_facts:
#         return [], 1.0
#     present_facts = sum(1 for fact in inferred_facts if any(fact in hop for hop in hops))
#     compliance_rate = present_facts / len(inferred_facts)
#     return inferred_facts, compliance_rate

# # Compute F1 score
# def compute_f1(pred_answer: str, true_answer: str) -> float:
#     pred_tokens = set(pred_answer.lower().split())
#     true_tokens = set(true_answer.lower().split())
#     common = pred_tokens.intersection(true_tokens)
#     if not common:
#         return 0.0
#     precision = len(common) / len(pred_tokens)
#     recall = len(common) / len(true_tokens)
#     return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

# # Visualize results
# def visualize_results(metrics: Dict):
#     # Overall metrics bar plot
#     plt.figure(figsize=(10, 6))
#     overall_metrics = {
#         "Consistency": metrics["consistency_score"],
#         "Transitivity": metrics["transitivity_compliance_rate"],
#         "EM Score": metrics["em_score"],
#         "F1 Score": metrics["f1_score"],
#         "Reasoning Accuracy": metrics["reasoning_step_accuracy"]
#     }
#     sns.barplot(x=list(overall_metrics.values()), y=list(overall_metrics.keys()), palette="Blues_d")
#     plt.title("Overall Evaluation Metrics", fontsize=14)
#     plt.xlabel("Score", fontsize=12)
#     plt.xlim(0, 1)
#     for i, v in enumerate(overall_metrics.values()):
#         plt.text(v + 0.01, i, f"{v:.3f}", va="center")
#     plt.tight_layout()
#     plt.show()

#     # Temporal sensitivity bar plot
#     plt.figure(figsize=(8, 5))
#     temporal_metrics = metrics["temporal_sensitivity"]
#     sns.barplot(x=list(temporal_metrics.values()), y=list(temporal_metrics.keys()), palette="Greens_d")
#     plt.title("Temporal Sensitivity Scores by Factor", fontsize=14)
#     plt.xlabel("Score", fontsize=12)
#     plt.xlim(0, 1)
#     for i, v in enumerate(temporal_metrics.values()):
#         plt.text(v + 0.01, i, f"{v:.3f}", va="center")
#     plt.tight_layout()
#     plt.show()

#     # Pie chart for factor distribution
#     plt.figure(figsize=(6, 6))
#     factor_counts = metrics["factor_counts"]
#     plt.pie(factor_counts.values(), labels=factor_counts.keys(), autopct='%1.1f%%', colors=sns.color_palette("Pastel1"))
#     plt.title("Dataset Factor Distribution", fontsize=14)
#     plt.tight_layout()
#     plt.show()

# # Main evaluation function
# def evaluate_menatqa(dataset: List[Dict]):
#     metrics = {
#         "consistency_score": 0.0,
#         "transitivity_compliance_rate": 0.0,
#         "em_score": 0.0,
#         "f1_score": 0.0,
#         "reasoning_step_accuracy": 0.0,
#         "temporal_sensitivity": {"scope": 0.0, "order": 0.0, "counterfactual": 0.0},
#         "total_samples": len(dataset),
#         "factor_counts": {"scope": 0, "order": 0, "counterfactual": 0}
#     }
    
#     consistent_pairs = 0
#     total_transitivity_rate = 0.0
#     em_correct = 0
#     total_f1 = 0.0
#     total_hops = 0
#     correct_hops = 0
    
#     for sample in tqdm(dataset, desc="Evaluating MenatQA"):
#         question = sample["question"]
#         context = sample["context"]
#         true_answer = sample["answer"]
#         factor = sample["factor"]
#         metrics["factor_counts"][factor] += 1
        
#         pred_answer, forward_hops = forward_reasoning(question, context)
#         backward_hops = backward_reasoning(question, context, pred_answer)
        
#         if check_consistency(forward_hops, backward_hops):
#             consistent_pairs += 1
        
#         inferred_facts, compliance_rate = transitivity_check(forward_hops)
#         total_transitivity_rate += compliance_rate
        
#         if pred_answer.strip().lower() == true_answer.strip().lower():
#             em_correct += 1
#             metrics["temporal_sensitivity"][factor] += 1
        
#         f1 = compute_f1(pred_answer, true_answer)
#         total_f1 += f1
        
#         total_hops += len(forward_hops)
#         correct_hops += len(forward_hops) if f1 > 0.5 else 0
    
#     metrics["consistency_score"] = consistent_pairs / len(dataset) if len(dataset) > 0 else 0.0
#     metrics["transitivity_compliance_rate"] = total_transitivity_rate / len(dataset) if len(dataset) > 0 else 0.0
#     metrics["em_score"] = em_correct / len(dataset) if len(dataset) > 0 else 0.0
#     metrics["f1_score"] = total_f1 / len(dataset) if len(dataset) > 0 else 0.0
#     metrics["reasoning_step_accuracy"] = correct_hops / total_hops if total_hops > 0 else 0.0
#     for factor in metrics["temporal_sensitivity"]:
#         if metrics["factor_counts"][factor] > 0:
#             metrics["temporal_sensitivity"][factor] /= metrics["factor_counts"][factor]
    
#     # Print results to console
#     print("\n=== Evaluation Results ===")
#     print(f"Total Samples: {metrics['total_samples']}")
#     print(f"Consistency Score: {metrics['consistency_score']:.3f}")
#     print(f"Transitivity Compliance Rate: {metrics['transitivity_compliance_rate']:.3f}")
#     print(f"EM Score: {metrics['em_score']:.3f}")
#     print(f"F1 Score: {metrics['f1_score']:.3f}")
#     print(f"Reasoning Step Accuracy: {metrics['reasoning_step_accuracy']:.3f}")
#     print("\nTemporal Sensitivity Scores:")
#     for factor, score in metrics["temporal_sensitivity"].items():
#         print(f"  {factor.capitalize()}: {score:.3f}")
#     print("\nFactor Distribution:")
#     for factor, count in metrics["factor_counts"].items():
#         print(f"  {factor.capitalize()}: {count} samples")
    
#     # Visualize results
#     visualize_results(metrics)
#     return metrics

# # Run evaluation
# if __name__ == "__main__":
#     try:
#         dataset = load_menatqa_dataset()
#         # For testing, you can use a smaller subset to reduce runtime
#         # dataset = dataset[:100]  # Uncomment to test with 100 samples
#         results = evaluate_menatqa(dataset)
#     except Exception as e:
#         logger.error(f"Evaluation failed: {e}")

2025-06-01 03:36:40,762 - WARNING - bitsandbytes not found. Running without 4-bit quantization.
2025-06-01 03:36:41,210 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
2025-06-01 03:36:41,688 - INFO - Loaded MenatQA dataset with 999 examples
Evaluating MenatQA:   0%|          | 0/999 [00:00<?, ?it/s]
2025-06-01 03:36:41,691 - ERROR - Evaluation failed: 'factor'


In [ ]:
import os
import json
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from typing import List, Dict, Tuple, Any, Set
from tqdm import tqdm
import random
import networkx as nx
import torch
from pathlib import Path  # Added this import
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# --- PROPER MENATQA DATA LOADING (from first code) ---
def load_menatqa_dataset(file_path='./MenatQA.json'):
    """Load MenatQA dataset from JSON file with automatic download."""
    file_path = Path(file_path)
    
    if not file_path.exists():
        import urllib.request
        print(f"Downloading MenatQA to {file_path}...")
        urllib.request.urlretrieve(
            "https://raw.githubusercontent.com/weiyifan1023/MenatQA/main/datasets/MenatQA.json",
            str(file_path)
        )
    
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(f"Loaded MenatQA dataset with {len(data)} examples")
    return data

# --- Model Setup ---

class QwenReasoningModel:
    def __init__(self, model_name="Qwen/Qwen3-0.6B", device="auto"):
        """Initialize Qwen model for reasoning tasks."""
        print(f"Loading {model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name, 
            trust_remote_code=True,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map=device
        )
        
        # Add padding token if not present
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            
        self.generator = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device_map=device,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            pad_token_id=self.tokenizer.eos_token_id
        )
        
        # Load semantic similarity model
        print("Loading semantic similarity model...")
        self.semantic_model = SentenceTransformer('all-MiniLM-L6-v2')
        
    def generate_text(self, prompt: str, max_length: int = 512) -> str:
        """Generate text using Qwen model."""
        try:
            result = self.generator(
                prompt,
                max_new_tokens=max_length,
                do_sample=True,
                temperature=0.7,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )
            generated_text = result[0]['generated_text']
            # Remove the input prompt from the generated text
            response = generated_text[len(prompt):].strip()
            return response
        except Exception as e:
            print(f"Error in text generation: {e}")
            return ""

# --- Forward vs. Backward Reasoning Analysis ---

def generate_forward_chain(question: str, model: QwenReasoningModel, context: str = "", max_length=512):
    """Generate forward reasoning chain: Q → (hop₁, hop₂, …) → A using Qwen model."""
    
    prompt = f"""Given the following question, provide a step-by-step reasoning chain that leads to the answer.

Question: {question}
{f"Context: {context}" if context else ""}

Please provide your reasoning in clear steps, where each step builds on the previous one:

Step-by-step reasoning:
1."""

    try:
        response = model.generate_text(prompt, max_length)
        
        # Parse the response into reasoning hops
        hops = []
        lines = response.split('\n')
        
        current_step = ""
        for line in lines:
            line = line.strip()
            if re.match(r'^\d+\.', line):  # New numbered step
                if current_step:
                    hops.append(current_step.strip())
                current_step = re.sub(r'^\d+\.', '', line).strip()
            elif line and not line.startswith('Answer:') and not line.startswith('Therefore:'):
                current_step += " " + line
            elif line.startswith('Answer:') or line.startswith('Therefore:'):
                if current_step:
                    hops.append(current_step.strip())
                # Extract final answer
                answer = line.replace('Answer:', '').replace('Therefore:', '').strip()
                return hops, answer
        
        if current_step:
            hops.append(current_step.strip())
            
        # Try to extract answer from the last hop or generate a simple one
        answer = hops[-1] if hops else "Unable to determine answer"
        
        return hops, answer
        
    except Exception as e:
        print(f"Error generating forward chain: {e}")
        # Fallback to simple reasoning
        return [f"Analyzing the question: {question}", "Processing available information", "Determining the answer"], "Answer pending"

def generate_backward_chain(answer: str, question: str, model: QwenReasoningModel, context: str = "", max_length=512):
    """Generate backward reasoning chain: A → (hop₁, hop₂, …) → Q using Qwen model."""
    
    prompt = f"""Given the following answer and question, work backwards to show the reasoning steps that led to this answer.

Answer: {answer}
Question: {question}
{f"Context: {context}" if context else ""}

Starting from the answer, trace back the logical steps that would lead to this conclusion:

Backward reasoning (from answer to question):
1. Starting with the answer: {answer}
2."""

    try:
        response = model.generate_text(prompt, max_length)
        
        # Parse the response into reasoning hops
        hops = []
        lines = response.split('\n')
        
        current_step = ""
        for line in lines:
            line = line.strip()
            if re.match(r'^\d+\.', line):  # New numbered step
                if current_step:
                    hops.append(current_step.strip())
                current_step = re.sub(r'^\d+\.', '', line).strip()
            elif line and not line.lower().startswith('question:'):
                current_step += " " + line
            elif line.lower().startswith('question:'):
                if current_step:
                    hops.append(current_step.strip())
                break
                
        if current_step:
            hops.append(current_step.strip())
            
        return hops
        
    except Exception as e:
        print(f"Error generating backward chain: {e}")
        # Fallback reasoning
        return [f"The answer '{answer}' was determined", "This required analyzing the question", f"The original question was: {question}"]

def compute_semantic_similarity(text1: str, text2: str, semantic_model: SentenceTransformer) -> float:
    """Compute semantic similarity between two texts using sentence transformers."""
    try:
        embeddings = semantic_model.encode([text1, text2])
        similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
        return float(similarity)
    except Exception as e:
        print(f"Error computing semantic similarity: {e}")
        return 0.0

def evaluate_forward_backward_consistency(df: pd.DataFrame, model: QwenReasoningModel) -> Dict:
    """Evaluate consistency between forward and backward reasoning chains using semantic similarity."""
    consistency_scores = []
    hop_consistency_matrix = []
    total_evaluated = 0
    detailed_examples = []

    # Use all data points instead of sampling
    for idx in tqdm(range(len(df)), desc="Evaluating F/B Consistency"):
        try:
            row = df.iloc[idx]
            question = row['question']
            context = row.get('context', '') or row.get('passage', '') or ''

            # Generate forward reasoning chain
            forward_hops, forward_answer = generate_forward_chain(question, model, context)
            
            if not forward_hops:
                continue

            # Generate backward reasoning chain
            backward_hops = generate_backward_chain(forward_answer, question, model, context)
            
            if not backward_hops:
                continue

            # Check consistency - compare each backward hop with corresponding forward hop in reverse
            hop_consistencies = []
            min_hops = min(len(forward_hops), len(backward_hops))

            for i in range(min_hops):
                forward_idx = len(forward_hops) - 1 - i
                backward_idx = i

                if forward_idx >= 0 and backward_idx < len(backward_hops):
                    similarity = compute_semantic_similarity(
                        forward_hops[forward_idx], 
                        backward_hops[backward_idx], 
                        model.semantic_model
                    )
                    hop_consistencies.append(similarity)

            # Overall consistency for this example
            if hop_consistencies:
                avg_consistency = sum(hop_consistencies) / len(hop_consistencies)
                consistency_scores.append(avg_consistency)
                hop_consistency_matrix.append(hop_consistencies)
                total_evaluated += 1
                
                # Store detailed example (only first 5 for memory efficiency)
                if len(detailed_examples) < 5:
                    detailed_examples.append({
                        'question': question,
                        'forward_hops': forward_hops,
                        'backward_hops': backward_hops,
                        'hop_similarities': hop_consistencies,
                        'avg_consistency': avg_consistency
                    })

        except Exception as e:
            print(f"Error processing example {idx}: {e}")
            continue

    # Calculate overall consistency score
    overall_consistency = sum(consistency_scores) / len(consistency_scores) if consistency_scores else 0

    # Calculate consistency by hop position
    hop_position_consistency = defaultdict(list)
    for consistencies in hop_consistency_matrix:
        for i, score in enumerate(consistencies):
            hop_position_consistency[i].append(score)

    avg_hop_position_consistency = {
        pos: sum(scores) / len(scores) if scores else 0
        for pos, scores in hop_position_consistency.items()
    }

    return {
        'overall_consistency': overall_consistency,
        'by_hop_position': avg_hop_position_consistency,
        'total_evaluated': total_evaluated,
        'detailed_examples': detailed_examples
    }

def evaluate_transitivity(df: pd.DataFrame, model: QwenReasoningModel) -> Dict:
    """Evaluate transitivity in reasoning chains with semantic similarity."""
    total_evaluated = 0
    transitivity_scores = []
    detailed_examples = []

    # Use all data points instead of sampling
    for idx in tqdm(range(len(df)), desc="Evaluating Transitivity"):
        try:
            row = df.iloc[idx]
            question = row['question']
            context = row.get('context', '') or row.get('passage', '') or ''

            # Generate reasoning hops using the model
            forward_hops, _ = generate_forward_chain(question, model, context)
            
            if len(forward_hops) < 3:  # Need at least 3 hops for transitivity
                continue

            # Extract facts from each hop
            all_facts = []
            hop_facts = []
            
            for hop in forward_hops:
                facts = extract_facts_from_hop(hop)
                all_facts.extend(facts)
                hop_facts.append(facts)

            # Skip if not enough facts for transitivity
            if len(all_facts) < 2:
                continue

            # Compute transitive closure
            inferred_facts = compute_transitive_closure(all_facts)

            # Skip if no inferred facts
            if not inferred_facts:
                continue

            # Check if inferred facts are contained in later hops
            facts_found = 0
            facts_total = len(inferred_facts)
            fact_details = []

            for fact in inferred_facts:
                found_in_hops = []
                # Check each hop after the first one
                for i, hop in enumerate(forward_hops[1:], 1):
                    if fact_contained_in_hop(fact, hop, model.semantic_model):
                        found_in_hops.append(i)
                        
                if found_in_hops:
                    facts_found += 1
                    
                fact_details.append({
                    'fact': f"{fact['subject']} {fact['relation']} {fact['object']}",
                    'found_in_hops': found_in_hops,
                    'category': fact.get('category', 'general')
                })

            transitivity_score = facts_found / facts_total if facts_total > 0 else 0
            transitivity_scores.append(transitivity_score)
            total_evaluated += 1
            
            # Store detailed example (only first 5 for memory efficiency)
            if len(detailed_examples) < 5:
                detailed_examples.append({
                    'question': question,
                    'hops': forward_hops,
                    'extracted_facts': [f"{f['subject']} {f['relation']} {f['object']}" for f in all_facts],
                    'inferred_facts': fact_details,
                    'transitivity_score': transitivity_score
                })

        except Exception as e:
            print(f"Error processing transitivity for example {idx}: {e}")
            continue

    # Calculate overall transitivity score
    overall_transitivity = sum(transitivity_scores) / len(transitivity_scores) if transitivity_scores else 0

    return {
        'overall_transitivity': overall_transitivity,
        'total_evaluated': total_evaluated,
        'scores': transitivity_scores,
        'detailed_examples': detailed_examples
    }

# --- Enhanced Fact Extraction for MenatQA ---

def extract_facts_from_hop(hop: str) -> List[Dict]:
    """Extract facts from a reasoning hop with patterns specific to MenatQA."""
    facts = []
    
    # MenatQA specific patterns for Middle Eastern and North African context
    
    # Geographic relationships
    geo_patterns = [
        (r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+is\s+(?:located\s+)?in\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)', 'located_in'),
        (r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+borders\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)', 'borders'),
        (r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+is\s+(?:the\s+)?capital\s+of\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)', 'capital_of'),
    ]
    
    # Historical relationships
    hist_patterns = [
        (r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+(?:ruled|conquered|controlled)\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)', 'ruled'),
        (r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+was\s+(?:founded|established)\s+(?:in\s+)?(\d+)', 'founded_in'),
        (r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+(?:occurred|happened)\s+(?:in\s+)?(\d+)', 'occurred_in'),
    ]
    
    # Cultural/Religious relationships
    cultural_patterns = [
        (r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+(?:speaks|uses)\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)', 'speaks'),
        (r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+(?:follows|practices)\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)', 'practices'),
        (r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+is\s+(?:a\s+)?([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+(?:country|nation|state)', 'is_type'),
    ]
    
    # Numerical relationships
    numerical_patterns = [
        (r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+has\s+(?:a\s+)?population\s+of\s+([0-9,]+)', 'population'),
        (r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+covers\s+([0-9,]+)', 'area'),
        (r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+is\s+(larger|smaller|bigger)\s+than\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)', 'size_comparison'),
    ]
    
    # Apply all pattern groups
    all_patterns = [
        (geo_patterns, 'geographic'),
        (hist_patterns, 'historical'), 
        (cultural_patterns, 'cultural'),
        (numerical_patterns, 'numerical')
    ]
    
    for pattern_group, category in all_patterns:
        for pattern, relation in pattern_group:
            matches = re.finditer(pattern, hop, re.IGNORECASE)
            for match in matches:
                groups = match.groups()
                if len(groups) >= 2:
                    facts.append({
                        'subject': groups[0].strip(),
                        'relation': relation,
                        'object': groups[1].strip(),
                        'category': category
                    })
    
    # General is/was patterns
    general_patterns = re.finditer(r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s+(?:is|was|are|were)\s+([^,.;]+)', hop)
    for match in general_patterns:
        facts.append({
            'subject': match.group(1).strip(),
            'relation': 'is',
            'object': match.group(2).strip(),
            'category': 'general'
        })
    
    return facts

def compute_transitive_closure(facts: List[Dict]) -> List[Dict]:
    """Compute transitive closure of facts with MenatQA-specific rules."""
    inferred_facts = []
    
    # Create a directed graph to represent relationships
    G = nx.DiGraph()
    
    # Add facts as edges in the graph with relation types
    for fact in facts:
        G.add_edge(fact['subject'], fact['object'], 
                   relation=fact['relation'], 
                   category=fact.get('category', 'general'))
    
    # Apply transitivity rules specific to geographical and historical relationships
    for u in G.nodes():
        for v in G.nodes():
            if u != v and G.has_edge(u, v):
                for w in G.nodes():
                    if v != w and w != u and G.has_edge(v, w):
                        uv_relation = G[u][v]['relation']
                        vw_relation = G[v][w]['relation']
                        
                        # Geographic transitivity
                        if uv_relation == 'located_in' and vw_relation == 'located_in':
                            # If A is in B and B is in C, then A is in C
                            inferred_facts.append({
                                'subject': u,
                                'relation': 'located_in',
                                'object': w,
                                'category': 'geographic',
                                'inferred': True
                            })
                        
                        # Historical transitivity  
                        elif uv_relation == 'ruled' and vw_relation == 'ruled':
                            # If A ruled B and B ruled C, then A influenced C
                            inferred_facts.append({
                                'subject': u,
                                'relation': 'influenced',
                                'object': w,
                                'category': 'historical',
                                'inferred': True
                            })
                        
                        # Identity transitivity
                        elif uv_relation == 'is' and vw_relation == 'is':
                            inferred_facts.append({
                                'subject': u,
                                'relation': 'is',
                                'object': w,
                                'category': 'general',
                                'inferred': True
                            })
                        
                        # Size comparison transitivity
                        elif uv_relation == 'larger' and vw_relation == 'larger':
                            inferred_facts.append({
                                'subject': u,
                                'relation': 'larger',
                                'object': w,
                                'category': 'numerical',
                                'inferred': True
                            })
    
    return inferred_facts

def fact_contained_in_hop(fact: Dict, hop: str, semantic_model: SentenceTransformer) -> bool:
    """Check if a fact is mentioned in a reasoning hop using semantic similarity."""
    # Create fact statement
    fact_statement = f"{fact['subject']} {fact['relation']} {fact['object']}"
    
    # Basic keyword matching
    subject_present = fact['subject'].lower() in hop.lower()
    object_present = fact['object'].lower() in hop.lower()
    
    if subject_present and object_present:
        # Both entities present - check semantic similarity for the relationship
        try:
            similarity = compute_semantic_similarity(fact_statement, hop, semantic_model)
            return similarity > 0.6  # Threshold for semantic similarity
        except:
            return True  # Fallback to basic presence check
    
    # If basic keyword matching fails, try semantic similarity
    try:
        similarity = compute_semantic_similarity(fact_statement, hop, semantic_model)
        return similarity > 0.7  # Higher threshold when keywords not directly present
    except:
        return False

def evaluate_transitivity(df: pd.DataFrame, model: QwenReasoningModel) -> Dict:
    """Evaluate transitivity in reasoning chains with semantic similarity."""
    total_evaluated = 0
    transitivity_scores = []
    detailed_examples = []

    # Sample subset for analysis
    sample_size = min(50, len(df))
    sample_indices = random.sample(range(len(df)), sample_size)

    for idx in tqdm(sample_indices, desc="Evaluating Transitivity"):
        try:
            row = df.iloc[idx]
            question = row['question']
            context = row.get('context', '') or row.get('passage', '') or ''

            # Generate reasoning hops using the model
            forward_hops, _ = generate_forward_chain(question, model, context)
            
            if len(forward_hops) < 3:  # Need at least 3 hops for transitivity
                continue

            # Extract facts from each hop
            all_facts = []
            hop_facts = []
            
            for hop in forward_hops:
                facts = extract_facts_from_hop(hop)
                all_facts.extend(facts)
                hop_facts.append(facts)

            # Skip if not enough facts for transitivity
            if len(all_facts) < 2:
                continue

            # Compute transitive closure
            inferred_facts = compute_transitive_closure(all_facts)

            # Skip if no inferred facts
            if not inferred_facts:
                continue

            # Check if inferred facts are contained in later hops
            facts_found = 0
            facts_total = len(inferred_facts)
            fact_details = []

            for fact in inferred_facts:
                found_in_hops = []
                # Check each hop after the first one
                for i, hop in enumerate(forward_hops[1:], 1):
                    if fact_contained_in_hop(fact, hop, model.semantic_model):
                        found_in_hops.append(i)
                        
                if found_in_hops:
                    facts_found += 1
                    
                fact_details.append({
                    'fact': f"{fact['subject']} {fact['relation']} {fact['object']}",
                    'found_in_hops': found_in_hops,
                    'category': fact.get('category', 'general')
                })

            transitivity_score = facts_found / facts_total if facts_total > 0 else 0
            transitivity_scores.append(transitivity_score)
            total_evaluated += 1
            
            # Store detailed example
            detailed_examples.append({
                'question': question,
                'hops': forward_hops,
                'extracted_facts': [f"{f['subject']} {f['relation']} {f['object']}" for f in all_facts],
                'inferred_facts': fact_details,
                'transitivity_score': transitivity_score
            })

        except Exception as e:
            print(f"Error processing transitivity for example {idx}: {e}")
            continue

    # Calculate overall transitivity score
    overall_transitivity = sum(transitivity_scores) / len(transitivity_scores) if transitivity_scores else 0

    return {
        'overall_transitivity': overall_transitivity,
        'total_evaluated': total_evaluated,
        'scores': transitivity_scores,
        'detailed_examples': detailed_examples[:5]  # Keep first 5 for inspection
    }

# --- Enhanced Visualization Functions ---

def visualize_consistency_results(consistency_results: Dict, save_path=None):
    """Visualize forward-backward consistency results with semantic evaluation."""
    overall = consistency_results['overall_consistency']
    by_position = consistency_results['by_hop_position']

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # Plot by hop position
    positions = sorted(by_position.keys())
    scores = [by_position[p] for p in positions]

    ax1.bar([str(p+1) for p in positions], scores, color='#3498db', alpha=0.8)
    ax1.axhline(y=overall, color='r', linestyle='--', label=f'Overall: {overall:.3f}')
    ax1.set_xlabel('Hop Position', fontsize=12)
    ax1.set_ylabel('Semantic Consistency Score', fontsize=12)
    ax1.set_title('Forward-Backward Consistency by Hop Position', fontsize=14)
    ax1.set_ylim(0, 1.05)
    ax1.legend()
    ax1.grid(axis='y', linestyle='--', alpha=0.7)

    # Distribution of consistency scores
    if 'detailed_examples' in consistency_results:
        all_similarities = []
        for example in consistency_results['detailed_examples']:
            all_similarities.extend(example.get('hop_similarities', []))
        
        if all_similarities:
            ax2.hist(all_similarities, bins=20, color='#2ecc71', alpha=0.7, edgecolor='black')
            ax2.axvline(x=np.mean(all_similarities), color='r', linestyle='--', 
                       label=f'Mean: {np.mean(all_similarities):.3f}')
            ax2.set_xlabel('Semantic Similarity Score', fontsize=12)
            ax2.set_ylabel('Frequency', fontsize=12)
            ax2.set_title('Distribution of Hop-wise Semantic Similarities', fontsize=14)
            ax2.legend()
            ax2.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Consistency visualization saved to {save_path}")
    plt.show()

def visualize_transitivity_results(transitivity_results: Dict, save_path=None):
    """Visualize transitivity results with category breakdown."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Overall transitivity score
    overall = transitivity_results['overall_transitivity']
    scores = transitivity_results['scores']
    
    # Score distribution
    ax1.hist(scores, bins=15, color='#e74c3c', alpha=0.7, edgecolor='black')
    ax1.axvline(x=overall, color='blue', linestyle='--', label=f'Mean: {overall:.3f}')
    ax1.set_xlabel('Transitivity Score', fontsize=12)
    ax1.set_ylabel('Frequency', fontsize=12)
    ax1.set_title('Distribution of Transitivity Scores', fontsize=14)
    ax1.legend()
    ax1.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Category breakdown if available
    if 'detailed_examples' in transitivity_results:
        category_counts = defaultdict(int)
        category_found = defaultdict(int)
        
        for example in transitivity_results['detailed_examples']:
            for fact_detail in example.get('inferred_facts', []):
                category = fact_detail.get('category', 'general')
                category_counts[category] += 1
                if fact_detail.get('found_in_hops'):
                    category_found[category] += 1
        
        categories = list(category_counts.keys())
        if categories:
            success_rates = [category_found[cat] / category_counts[cat] if category_counts[cat] > 0 else 0 
                           for cat in categories]
            
            bars = ax2.bar(categories, success_rates, color=['#9b59b6', '#f39c12', '#1abc9c', '#34495e'], alpha=0.8)
            ax2.set_ylabel('Transitivity Success Rate', fontsize=12)
            ax2.set_title('Transitivity by Fact Category', fontsize=14)
            ax2.set_ylim(0, 1.05)
            
            # Add value labels on bars
            for bar, rate in zip(bars, success_rates):
                height = bar.get_height()
                ax2.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                        f'{rate:.2f}', ha='center', va='bottom')
            
            ax2.grid(axis='y', linestyle='--', alpha=0.7)
            plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Transitivity visualization saved to {save_path}")
    plt.show()

def visualize_combined_results(consistency_results: Dict, transitivity_results: Dict, save_path=None):
    """Visualize combined consistency and transitivity results."""
    fig, ax = plt.subplots(figsize=(10, 6))

    # Data for plotting
    metrics = ['F-B Consistency\n(Semantic)', 'Transitivity\n(Graph-based)']
    values = [
        consistency_results['overall_consistency'],
        transitivity_results['overall_transitivity']
    ]
    colors = ['#3498db', '#2ecc71']

    # Create bar plot
    bars = ax.bar(metrics, values, color=colors, alpha=0.8, width=0.6)

    # Add value labels
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{value:.3f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

    ax.set_ylabel('Score', fontsize=14)
    ax.set_title('MenatQA Logical Consistency & Transitivity Evaluation\n(Qwen/Qwen3-0.6B + Semantic Analysis)', fontsize=16)
    ax.set_ylim(0, 1.05)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Add evaluation details as text
    eval_text = f"Evaluated: {consistency_results['total_evaluated']} examples (Consistency)\n"
    eval_text += f"Evaluated: {transitivity_results['total_evaluated']} examples (Transitivity)"
    ax.text(0.02, 0.98, eval_text, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Combined visualization saved to {save_path}")
    plt.show()

# --- Main Execution Functions ---

def evaluate_logical_consistency(df: pd.DataFrame, model: QwenReasoningModel) -> Dict:
    """Run complete logical consistency and transitivity evaluation with Qwen model."""
    print("\n1. Evaluating Forward-Backward Consistency with Semantic Analysis...")
    consistency_results = evaluate_forward_backward_consistency(df, model)

    print(f"\nForward-Backward Consistency Results:")
    print(f"Overall Semantic Consistency: {consistency_results['overall_consistency']:.4f}")
    print(f"Total examples evaluated: {consistency_results['total_evaluated']}")
    print("\nConsistency by Hop Position:")
    for pos, score in sorted(consistency_results['by_hop_position'].items()):
        print(f"Hop {pos+1}: {score:.4f}")

    print("\n2. Evaluating Transitivity with Graph-based Analysis...")
    transitivity_results = evaluate_transitivity(df, model)

    print(f"\nTransitivity Results:")
    print(f"Overall Transitivity: {transitivity_results['overall_transitivity']:.4f}")
    print(f"Total examples evaluated: {transitivity_results['total_evaluated']}")

    # Show sample detailed analysis
    if transitivity_results['detailed_examples']:
        print(f"\nSample Transitivity Analysis:")
        example = transitivity_results['detailed_examples'][0]
        print(f"Question: {example['question'][:100]}...")
        print(f"Inferred Facts Found: {len([f for f in example['inferred_facts'] if f['found_in_hops']])}/{len(example['inferred_facts'])}")

    # Combine results
    combined_results = {
        'consistency': consistency_results,
        'transitivity': transitivity_results,
        'model_info': {
            'model_name': 'Qwen/Qwen3-0.6B',
            'semantic_model': 'Qwen/Qwen3-0.6B',
            'evaluation_type': 'semantic_similarity'
        }
    }

    return combined_results

def main():
    """Run the complete logical consistency and transitivity evaluation pipeline with Qwen model."""
    print("=" * 60)
    print("MenatQA Logical Consistency & Transitivity Evaluation")
    print("Using Qwen/Qwen3-0.6B + Semantic Analysis")
    print("=" * 60)

    # Initialize model
    print("\n1. Initializing Qwen Model...")
    try:
        qwen_model = QwenReasoningModel("Qwen/Qwen3-0.6B")
    except Exception as e:
        print(f"Error loading model: {e}")
        print("Falling back to alternative model...")
        qwen_model = QwenReasoningModel("Qwen/Qwen3-0.6B")

    # Load MenatQA dataset using the proper function
    print("\n2. Loading MenatQA dataset...")
    try:
        menatqa_data = load_menatqa_dataset('./MenatQA.json')  # Now uses the proper function
        df = pd.DataFrame(menatqa_data)
        print(f"Successfully loaded {len(df)} examples")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        # Create sample data for testing if download fails
        df = pd.DataFrame([
            {'question': 'What is the capital of Egypt?', 'answer': 'Cairo'},
            {'question': 'Which country borders Jordan to the south?', 'answer': 'Saudi Arabia'},
            {'question': 'When was the Ottoman Empire founded?', 'answer': '1299'},
        ])
        print(f"Using sample data with {len(df)} examples")

    # Create results directory
    results_dir = './logical_consistency_results'
    os.makedirs(results_dir, exist_ok=True)

    # Evaluate logical consistency and transitivity
    print(f"\n3. Evaluating Logical Consistency and Transitivity on {len(df)} questions...")
    results = evaluate_logical_consistency(df, qwen_model)

    # Generate visualizations
    print("\n4. Generating Visualizations...")

    # Consistency visualization
    consistency_viz_path = f"{results_dir}/semantic_consistency_analysis.png"
    visualize_consistency_results(results['consistency'], save_path=consistency_viz_path)

    # Transitivity visualization
    transitivity_viz_path = f"{results_dir}/transitivity_analysis.png"
    visualize_transitivity_results(results['transitivity'], save_path=transitivity_viz_path)

    # Combined visualization
    combined_viz_path = f"{results_dir}/combined_logical_metrics.png"
    visualize_combined_results(results['consistency'], results['transitivity'], save_path=combined_viz_path)

    # Save detailed results
    print("\n5. Saving Detailed Results...")
    
    # Convert to serializable format
    serializable_results = {
        'model_info': results['model_info'],
        'consistency': {
            'overall_consistency': float(results['consistency']['overall_consistency']),
            'by_hop_position': {int(k): float(v) for k, v in results['consistency']['by_hop_position'].items()},
            'total_evaluated': int(results['consistency']['total_evaluated'])
        },
        'transitivity': {
            'overall_transitivity': float(results['transitivity']['overall_transitivity']),
            'total_evaluated': int(results['transitivity']['total_evaluated']),
            'score_distribution': {
                'mean': float(np.mean(results['transitivity']['scores'])) if results['transitivity']['scores'] else 0,
                'std': float(np.std(results['transitivity']['scores'])) if results['transitivity']['scores'] else 0
            }
        }
    }

    # Save results as JSON
    with open(f"{results_dir}/semantic_evaluation_results.json", 'w') as f:
        json.dump(serializable_results, f, indent=2)

    # Save detailed examples for inspection
    detailed_analysis = {
        'consistency_examples': results['consistency'].get('detailed_examples', []),
        'transitivity_examples': results['transitivity'].get('detailed_examples', [])
    }
    
    with open(f"{results_dir}/detailed_examples_analysis.json", 'w') as f:
        json.dump(detailed_analysis, f, indent=2)

    print(f"\n6. Evaluation Summary:")
    print(f"   • Semantic Consistency Score: {results['consistency']['overall_consistency']:.4f}")
    print(f"   • Graph-based Transitivity Score: {results['transitivity']['overall_transitivity']:.4f}")
    print(f"   • Model Used: {results['model_info']['model_name']}")
    print(f"   • Semantic Model: {results['model_info']['semantic_model']}")
    print(f"   • Results saved to: {results_dir}")

    print("\n" + "=" * 60)
    print("Evaluation Complete!")
    print("=" * 60)

    return results

# Run the evaluation
if __name__ == "__main__":
    results = main()

MenatQA Logical Consistency & Transitivity Evaluation
Using Qwen/Qwen3-0.6B + Semantic Analysis

1. Initializing Qwen Model...
Loading Qwen/Qwen3-0.6B...


2025-06-01 06:03:56,723 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Device set to use cuda:0
2025-06-01 06:03:57,087 - INFO - Use pytorch device_name: cuda:0
2025-06-01 06:03:57,087 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Loading semantic similarity model...

2. Loading MenatQA dataset...
Loaded MenatQA dataset with 999 examples
Successfully loaded 999 examples

3. Evaluating Logical Consistency and Transitivity on 999 questions...

1. Evaluating Forward-Backward Consistency with Semantic Analysis...


Batches: 100%|██████████| 1/1 [00:00<00:00, 194.78it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 217.07it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 217.24it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 216.79it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 184.45it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 189.16it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 211.58it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 202.88it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 189.33it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 199.05it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 204.38it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 204.44it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 192.61it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 195.27it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 196.53it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 205.86it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 203.37it/s]

Batches: 100%|██████████| 1/1 [